# How to identify / highlight differences between two versions of the same Dataset over time.

### See below an example with the Sales Dataframe (two elements are changed from Day1 to Day2):

In [17]:
import numpy as np
import pandas as pd

In [5]:
sales1 = pd.read_csv("Data1/Course_Material_Part2/Video_Lecture_NBs/sales.csv", index_col="Unnamed: 0")

In [6]:
sales1

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,27,45.0,7


In [7]:
sales1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, Steven to Paul
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mon     4 non-null      int64  
 1   Tue     4 non-null      int64  
 2   Wed     4 non-null      int64  
 3   Thu     3 non-null      float64
 4   Fri     4 non-null      int64  
dtypes: float64(1), int64(4)
memory usage: 192.0+ bytes


### Let make a copy of sales1

In [8]:
sales2 = sales1.copy()

In [9]:
sales2

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,27,45.0,7


### Change two elements in sales 2

In [10]:
sales2.iloc[0, 1] = 100

In [11]:
sales2.iloc[3,2] = 200

In [12]:
sales2

,Mon,Tue,Wed,Thu,Fri
Steven,34,100,15,NaN,33
Mike,45,9,74,87.0,12
Andi,17,33,54,8.0,29
Paul,87,67,200,45.0,7


### Compare for equality using ==

In [13]:
sales2 == sales1 #This gives us False for changed values/ or where the difference occurs

,Mon,Tue,Wed,Thu,Fri
Steven,True,False,True,False,True
Mike,True,True,True,True,True
Andi,True,True,True,True,True
Paul,True,True,False,True,True


In [15]:
sales1 == sales2

,Mon,Tue,Wed,Thu,Fri
Steven,True,False,True,False,True
Mike,True,True,True,True,True
Andi,True,True,True,True,True
Paul,True,True,False,True,True


### Let further inspect with the where () - Gives NAN for similar value and the actual value where the diff occurs

In [14]:
sales1.where(~(sales1 == sales2))#Returns where condition is true replace with NAN else keep the original value

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,27.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andi,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,27.0,NaN,NaN


In [16]:
sales2.where(~(sales2 == sales1))#Returns where condition is true replace with NAN else keep the original value

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,100.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andi,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,200.0,NaN,NaN


### For Only Highlighting the difference, We must concatenate the two datasets

In [18]:
sales_comp = pd.concat([sales1, sales2], axis=1, keys=["Day1_Sales", "Day2_Sales"])

In [19]:
sales_comp

Day1_Sales                   Day2_Sales                    
              Mon Tue Wed   Thu Fri        Mon  Tue  Wed   Thu Fri
Steven         34  27  15   NaN  33         34  100   15   NaN  33
Mike           45   9  74  87.0  12         45    9   74  87.0  12
Andi           17  33  54   8.0  29         17   33   54   8.0  29
Paul           87  67  27  45.0   7         87   67  200  45.0   7

## Now let highlight the difference

In [22]:
def highlight_diff(data, color="yellow"):
    attr = 'background-color: {}'.format(color)
    other = data.xs("Day1_Sales", axis="columns", level=-2)
    
    return pd.DataFrame(np.where(data.ne(other, level=1), attr, ''), index=data.index, columns=data.columns)

In [25]:
#Highlight the diff
sales_comp.style.apply(highlight_diff, axis=None)

## Now let print out the difference

In [28]:
def diff_pd(df1, df2):
    """ Identify difference between to pandas dataframes and print out the diff """
    assert (df1.columns == df2.columns).all(), \
           "DataFrame Columns are Different"
    
    if any(df1.dtypes != df2.dtypes):
        "Data types are different, trying to convert"
        df2 = df2.astype(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        #Need to account for np.nan != np.nan returning True
        diff_mask = (df1 != df2) & ~(df1.isnull() & df2.isnull())
        ne_stacked = diff_mask.stack()
        changed = ne_stacked[ne_stacked]
        changed.index.names = ["id", "col"]
        difference_locations = np.where(diff_mask)
        changed_from = df1.values[difference_locations]
        changed_to = df2.values[difference_locations]
        
        return pd.DataFrame({"From": changed_from, "To" : changed_to}, index=changed.index)

In [29]:
#Printing the difference columns and rows occurence
diff_pd(sales1, sales2)

,,From,To
id,col,,
Steven,Tue,27.0,100.0
Paul,Wed,27.0,200.0
